In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords

# Eğer stopwords corpusu henüz indirilmediyse indirin
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emirh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [2]:
df = pd.read_csv("NLPlabeledData.tsv",delimiter="\t",quoting=3)

In [3]:
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [4]:
len(df)

25000

In [5]:
len(df["review"])

25000

In [10]:
# simdi veri temizleme islemine gecelim yp nin anlamasi icin, metindeki harflerin hepsi kucuk harf yapilacak
# noktalama isaretleri temizlenecek ve sayilar ile anlamsiz kelimeler cikarilacak
# (anlamsiz kelimelerden kasit inglizcedeki is are gibi tek basina anlami olmayan kelimeler "stop word" diye gecer ayrica)

def process(review):
    # review without html tags
    review = BeautifulSoup(review).get_text()
    # review without punctuation 
    review = re.sub("[^a-zA-Z]", ' ', review)
    # converting into lowercase and splitting to eliminate stop words
    review = review.lower()
    review = review.split()
    # review without stop words
    swords = set(stopwords.words("english"))
    review = [w for w in review if w not in swords]
    return (" ".join(review))
    

In [11]:
# bi usteki fonksiyon ile datayi temizliyoruz
# her 1000 review temizleme sonrasi ekrana yazdiyoruz ki ilermeyi gorebilelim

train_x_tum = []
for r in range(len(df["review"])):
    if (r+1)%1000 == 0:
        print("No of reviews processed ", r+1)
    train_x_tum.append(process(df["review"][r]))

C:\Users\emirh\AppData\Local\Temp\ipykernel_22192\310373298.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review = BeautifulSoup(review).get_text()


No of reviews processed  1000
No of reviews processed  2000
No of reviews processed  3000
No of reviews processed  4000
No of reviews processed  5000
No of reviews processed  6000
No of reviews processed  7000
No of reviews processed  8000
No of reviews processed  9000
No of reviews processed  10000
No of reviews processed  11000
No of reviews processed  12000
No of reviews processed  13000
No of reviews processed  14000
No of reviews processed  15000
No of reviews processed  16000
No of reviews processed  17000
No of reviews processed  18000
No of reviews processed  19000
No of reviews processed  20000
No of reviews processed  21000
No of reviews processed  22000
No of reviews processed  23000
No of reviews processed  24000
No of reviews processed  25000


In [12]:
# verileri bolelim egitim icin
x = train_x_tum
y = np.array(df["sentiment"])

train_x,test_x , y_train , y_text = train_test_split(x,y,  test_size = 0.1 , random_state = 42)

In [13]:
# sklearn deki countvectorizer fonksiyonu ile 5000 kelimelik bag of words olusturuyoruz
vectorizer = CountVectorizer(max_features = 5000)
# train verilemizi feature vektore matrisine ceviriyoruz
train_x = vectorizer.fit_transform(train_x)
# array a donuturuyoruz 
train_x = train_x.toarray()

In [14]:
# modeli olusturalim
train_y = y_train
model = RandomForestClassifier(n_estimators = 100)
model.fit(train_x,train_y)

RandomForestClassifier()

In [15]:
# simdi test datasini matrise cevirelim
test_xx = vectorizer.transform(test_x)
text_xx = test_xx.toarray()

In [18]:
# dogrulugumuza bakalim
test_prediction = model.predict(test_xx)
dogruluk = roc_auc_score(y_text, test_prediction)
print("Dogruluk orani ", dogruluk * 100)

Dogruluk orani  84.16495093520169
